In [9]:
import numpy as np
from prondict import *
from lab2_proto import *
from lab2_tools import *
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
phoneHMMs = phoneHMMs = np.load('lab2_models_onespkr.npz', allow_pickle=True)['phoneHMMs'].item()



In [10]:
isolated = {}
for digit in prondict.keys():
 isolated[digit] = ['sil'] + prondict[digit] + ['sil']


wordHMMs = {}
for digit in prondict.keys():
 wordHMMs[digit] = concatHMMs(phoneHMMs, isolated[digit])


In [11]:
example = np.load('lab2_example.npz', allow_pickle=True)['example'].item()


5.1 Gaussian emission probabilities


In [12]:
lmfcc= example['lmfcc'] 
example_obsloglik= example['obsloglik']
obsloglik = log_multivariate_normal_density_diag(X=lmfcc,means=wordHMMs['o']['means'],covars=wordHMMs['o']['covars'])
print(np.array_equal(np.round(example_obsloglik,4),np.round(obsloglik,4) ))



True


5.2 Forward Algorithm

In [13]:

logalpha = forward(obsloglik,np.log(wordHMMs['o']['startprob']),np.log(wordHMMs['o']['transmat'][:-1, :-1]))
print(np.array_equal(np.round(logalpha,4),np.round(example['logalpha'],4)))




True


5.3 Backward Algorithm

In [14]:
logbeta= backward(obsloglik,np.log(wordHMMs['o']['startprob']),np.log(wordHMMs['o']['transmat'][:-1, :-1]))
print(np.array_equal(np.round(logbeta,4),np.round(example['logbeta'],4)))

True


State posteriors (gamma)


In [15]:

log_gamma = statePosteriors(logalpha,logbeta)
print(np.array_equal(np.round(log_gamma,1) ,np.round(example['loggamma'],1)))

True


In [16]:
viterbi_loglik , path = viterbi(obsloglik,np.log(wordHMMs['o']['startprob']),np.log(wordHMMs['o']['transmat'][:-1, :-1]))
print(np.array_equal(np.round(viterbi_loglik,1) ,np.round(example['vloglik'],1)))


True
